In [1]:
from gat_mem_org_log.dataprocessor import *
from util import *
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RandomizedSearchCV 
from sklearn.model_selection import GridSearchCV, KFold
from gat_mem_org_log.log import *
def custom_scorer(y_true, y_pred, actual_scorer):
    score = np.nan

    try:
      score = -actual_scorer(y_true, y_pred)
    except Exception: 
      pass

    return score


from sklearn.metrics import make_scorer
rmse = make_scorer(custom_scorer, actual_scorer = mean_squared_error)

def get_variable_importance(pid2feature_importance):
    temp_list = []
    for pid in pid2feature_importance:
        temp_list.append(np.expand_dims(pid2feature_importance[pid], axis=0))
    return np.concatenate(temp_list, axis=0).mean(axis=0)

class XGB():
    def __init__(self, CONF):
        self.CONF = CONF
        self.pid2model = {}
        self.metrics_dic = {
            'rmse': [],
            'mape': [],
            'mae': [],
            'grmse': [],
            'time_lag' : [],
        }
        self.pid2feature_importance = {}
        self.pid2prediction = {}

    def __call__(self, data_proc,  pid, batch_size):
        seed = self.CONF['seed']
        random.seed(seed)
        np.random.seed(seed)
        os.environ['PYTHONHASHSEED'] = str(seed)


        
        mean, std = data_proc.get_mean_std(pid)
        # params = {
        #     'min_child_weight': [1, 5, 10],
        #     'gamma': [0.5, 1, 1.5, 2, 5],
        #     'subsample': [0.6, 0.8, 1.0],
        #     'colsample_bytree': [0.6, 0.8, 1.0],
        #     'max_depth': [3, 4, 5, 6, 7]
        #     'n_estimators': [50,100,200],
        #     'learning_rate': [0.01, 0.1, 1],
        # }
        # G, attris, time_attris, y, target_time = data_proc.get_all_train(device='cpu', seq_len=self.CONF['seq_len'], attri_list=self.CONF['attri_list'], time_attri_list=self.CONF['time_attri_list'], pid=pid,)
        # X = torch.cat([torch.unsqueeze(G, dim=2), attris, time_attris], dim=2).detach().cpu().numpy()
        # B, T, N = X.shape
        # X = X.reshape(B, -1)
        # y_norm = (y-mean)/std
        
        # xgb_cv = RandomizedSearchCV(xgb.XGBRegressor(objective="reg:squarederror", random_state=seed), params, cv=3, scoring = rmse, random_state=seed, n_iter=10,)
        # xgb_cv.fit(X, y_norm)
        
        # self.pid2model[pid] = xgb.XGBRegressor(**xgb_cv.best_params_)

        
        if pid not in self.pid2model:
            self.pid2model[pid] = xgb.XGBRegressor(
                # tree_method='gpu_hist', 
                objective="reg:squarederror", random_state=seed, min_child_weight=5, gamma=1, subsample=0.95, colsample_bytree=1.0, max_depth=5, n_estimators=50, learning_rate=0.1)
    
        
        G, attris, time_attris, y, target_time = data_proc.get_all_train(device='cpu', seq_len=self.CONF['seq_len'], attri_list=self.CONF['attri_list'], time_attri_list=self.CONF['time_attri_list'], pid=pid)
        X = torch.cat([torch.unsqueeze(G, dim=2), attris, time_attris], dim=2).detach().cpu().numpy()
        B, T, N = X.shape
        X = X.reshape(B, -1)
        y_norm = (y-mean)/std

        self.pid2model[pid].fit(X, y_norm)

        G, attris, time_attris, y, target_time, seq_st_ed = data_proc.get_test(device='cpu', seq_len=self.CONF['seq_len'], attri_list=self.CONF['attri_list'], time_attri_list=self.CONF['time_attri_list'], pid=pid)
        X = torch.cat([torch.unsqueeze(G, dim=2), attris, time_attris], dim=2).detach().cpu().numpy()
        B, T, N = X.shape
        X = X.reshape(B, -1)


        # self.pid2model[pid] = random_search
        pred = self.pid2model[pid].predict(X)


        predicted_np = pred*std + mean

        self.pid2prediction[pid] = predicted_np
        self.metrics_dic['rmse'].append(mean_squared_error(y, predicted_np)**0.5)
        self.metrics_dic['mape'].append(mean_absolute_percentage_error(y, predicted_np) * 100)
        self.metrics_dic['mae'].append(mean_absolute_error(y, predicted_np))
        self.metrics_dic['grmse'].append(cal_gmse(y, predicted_np[:, None]))
        self.metrics_dic['time_lag'].append(cal_time_lag(y, predicted_np[:, None], seq_st_ed, self.CONF['interval']))
        feature_importances_ = self.pid2model[pid].feature_importances_.reshape(T, N)
        
        self.pid2feature_importance[pid] = np.mean(np.abs(feature_importances_), axis=0)
        # colsample_bytree=1.0, gamma=1, max_depth=5, min_child_weight=5, subsample=1.0;, score=-0.176

class LR():
    def __init__(self, CONF):
        self.CONF = CONF
        self.pid2model = {}
        self.metrics_dic = {
            'rmse': [],
            'mape': [],
            'mae': [],
            'grmse': [],
            'time_lag' : [],
        }

        self.pid2feature_importance = {

        }
        self.pid2prediction = {}
        self.X1 = None
    def __call__(self, data_proc,  pid, batch_size=None):
        
        seed = self.CONF['seed']
        if batch_size is not None: 
            seed = 1
        random.seed(seed)
        np.random.seed(seed)
        os.environ['PYTHONHASHSEED'] = str(seed)



       
        if pid not in self.pid2model:
            self.pid2model[pid] = LinearRegression()
        
        mean, std = data_proc.get_mean_std(pid)

        G, attris, time_attris, y, target_time = data_proc.get_all_train(device='cpu', seq_len=self.CONF['seq_len'], attri_list=self.CONF['attri_list'], time_attri_list=self.CONF['time_attri_list'], pid=pid)
        G_np = G.detach().cpu().numpy()[..., None]
        attri_np = attris.detach().cpu().numpy()
        time_attris_np = time_attris.detach().cpu().numpy()
        X = np.concatenate([G_np, attri_np, time_attris_np], axis=-1)
        B, T, C = X.shape
        X = X.transpose(0, 2, 1) # B C T
        # X = X[:, 0, :]
        y_norm = (y-mean)/std
        X = X.reshape(B, -1)
        self.pid2model[pid].fit(X, y_norm)

        G, attris, time_attris, y, target_time, seq_st_ed = data_proc.get_test(device='cpu', seq_len=self.CONF['seq_len'], attri_list=self.CONF['attri_list'], time_attri_list=self.CONF['time_attri_list'], pid=pid)
        G_np = G.detach().cpu().numpy()[..., None]
        T = G_np.shape[1]
        attri_np = attris.detach().cpu().numpy()
        time_attris_np = time_attris.detach().cpu().numpy()
        X = np.concatenate([G_np, attri_np, time_attris_np], axis=-1)
        B, T, C = X.shape
        X = X.transpose(0, 2, 1) # B C T
        # X = X[:, 0, :]
        X = X.reshape(B, -1)
        pred = self.pid2model[pid].predict(X)
        predicted_np = pred*std + mean
        predicted_np = np.clip(predicted_np, 40, 400)



        self.pid2prediction[pid] = predicted_np
        self.metrics_dic['rmse'].append(mean_squared_error(y, predicted_np)**0.5)
        self.metrics_dic['mape'].append(mean_absolute_percentage_error(y, predicted_np) * 100)
        self.metrics_dic['mae'].append(mean_absolute_error(y, predicted_np))
        self.metrics_dic['grmse'].append(cal_gmse(y, predicted_np[:, None]))
        self.metrics_dic['time_lag'].append(cal_time_lag(y, predicted_np[:, None], seq_st_ed, self.CONF['interval']))

        feature_importances_ = self.pid2model[pid].coef_ # 
        feature_importances_ = feature_importances_.reshape(C, T)
        self.pid2feature_importance[pid] = np.abs(feature_importances_).mean(-1)


        

In [2]:

def print_metrics_dic(metrics_dic):

    for mtc in metrics_dic:
        print(f'{mtc}, {np.mean(metrics_dic[mtc]):.5f}, {np.std(metrics_dic[mtc]):.5f}')
    
    print(metrics_dic['rmse'])

In [3]:
def run_exp(dataset, version, method, seed, pred_window, n_prev, seq_len, attri_list, interval):


    CONF = {
        'data_path':f'../../code_data/{dataset}/{version}/',
        'log_path': f'../../code_log/{version}/{dataset}',
        'dataset': dataset,
        'n_prev' : n_prev,
        'pred_window' : pred_window,
        'seq_len': seq_len,
        'mix_test_and_valid_2018': False,

        'attri_list': attri_list,

        'time_attri_list' : ['timestamp'],     

        'model_name': method, 

        
        'comments': f'pred_{pred_window}_all_features_seed_{seed}',
        'save_model': True,
        
        'interval': interval,
        'log_epoch_vari_imp': False,
        'seed':seed,
    }
    log = Log(CONF['dataset'] + '_' + CONF['model_name'] + '_' + CONF['comments'], CONF)
    data_proc = DataProcessor(CONF)
    if method == 'xgboost':
        model = XGB(CONF)
    elif method == 'linear_regression':
        model = LR(CONF)

    for pid in data_proc.test_pid2data_npy:
        batch_size = 100 if dataset == 'shanghai_t1dm' and method == 'linear_regression' else None
        model(data_proc, pid, batch_size) 
    log.save_prediction(model.pid2prediction)
    log.save_metrics_dic(model.metrics_dic)
    if method == 'xgboost':
        log.save_xgboost_models(model.pid2model)
    elif method == 'linear_regression':
        log.save_scikit_models(model.pid2model)

    variable_importance = get_variable_importance(model.pid2feature_importance)
    log.summary_vari_importance(variable_importance, name=f'train_all_variable_importance')
    
    print_metrics_dic(model.metrics_dic)

# rmse, 22.30146, 2.75739
# mape, 10.95096, 2.14294
# mae, 15.98284, 1.94163
# grmse, 27.79454, 3.67998
# time_lag, -8.54861, 6.23855

In [4]:
for method in ['xgboost']:
    for dataset in ['ohio', 'arises', 'shanghai_t1dm', 'shanghai_t2dm']:
        for seed in [1, 10, 20, 100]:
            pred_window = 2 if 'shanghai' in dataset else 6
            n_prev = 16 if 'shanghai' in dataset else 48
            seq_len = n_prev
            version = 'nips_23'
            interval = 15 if 'shanghai' in dataset else 5
            if dataset == 'ohio':
                attri_list = [ 'meal', 'bolus', 'basal', 'finger_stick',
            'basis_skin_temperature', 'basis_gsr', 'basis_sleep', 'acceleration',
            'exercise', 'sleep', 'work', 'basis_steps', 'basis_heart_rate', 'basis_air_temperature']
            elif dataset == 'arises':
                attri_list = [
            'meal', 'bolus', 'basal', 'correction_bolus',
            'finger_stick', 'EDA', 'SCL', 'SCR', 'HR', 'TEMP', 'ACC', 'RMSSD', 'SDNN',
            'medianNNI', 'CVNNI', 'CVSD', 'pNNX', 'meanHR', 'minHR', 'maxHR', 'VLF',
            'LF', 'HF', 'LHR', ]
            elif dataset == 'shanghai_t1dm' or dataset == 'shanghai_t2dm':
                attri_list = ['finger_stick', 'blood_ketone', 'meal','insulin_dose_sc', 'hypo_agents', 'insulin_bolus', 'basal','insulin_iv']

            run_exp(dataset, version, method, seed, pred_window, n_prev, seq_len, attri_list, interval)

rmse, 22.53452, 3.33304
mape, 10.87684, 2.22624
mae, 16.04948, 2.27347
grmse, 28.99639, 4.78799
time_lag, -9.16667, 6.50854
[23.87624879137542, 20.36577598676193, 19.579901904599776, 26.533891993380962, 20.015382573180588, 22.605288860632953, 29.23158245113761, 17.991682038058414, 20.206331616657778, 24.745840628638355, 25.786623493041407, 19.475740650279597]
rmse, 22.47450, 3.33280
mape, 10.93782, 2.25187
mae, 16.07693, 2.26123
grmse, 28.80741, 4.78557
time_lag, -9.37500, 6.39078
[23.077674071000825, 20.437228233093705, 19.441186095423166, 26.998206184986277, 19.94233783634472, 22.56561274732218, 28.65542890896996, 17.971456997182017, 20.308322887811908, 25.082896012288558, 25.974250367025153, 19.23944991691469]
rmse, 22.56149, 3.37451
mape, 10.97082, 2.26064
mae, 16.15664, 2.27892
grmse, 28.95092, 4.86797
time_lag, -9.37500, 6.39078
[23.576214510073495, 20.4931822814206, 19.459759787226627, 27.377519326139193, 19.878052291173425, 22.76829916303244, 28.625941365184286, 18.058740349961

<Figure size 2000x1500 with 0 Axes>

<Figure size 2000x1500 with 0 Axes>

<Figure size 2000x1500 with 0 Axes>

<Figure size 2000x1500 with 0 Axes>

...

<Figure size 2000x1500 with 0 Axes>

<Figure size 2000x1500 with 0 Axes>

<Figure size 2000x1500 with 0 Axes>

<Figure size 2000x1500 with 0 Axes>

<Figure size 2000x1500 with 0 Axes>

In [5]:
for method in ['linear_regression']:
    for dataset in ['shanghai_t1dm', 'ohio', 'arises', 'shanghai_t2dm']:
        for seed in [1, 10, 20, 100]:
            print(method, dataset, seed)
            pred_window = 2 if 'shanghai' in dataset else 6
            n_prev = 16 if 'shanghai' in dataset else 48
            seq_len = n_prev
            version = 'nips_23'
            interval = 15 if 'shanghai' in dataset else 5
            if dataset == 'ohio':
                attri_list = [ 'meal', 'bolus', 'basal', 'finger_stick',
            'basis_skin_temperature', 'basis_gsr', 'basis_sleep', 'acceleration',
            'exercise', 'sleep', 'work', 'basis_steps', 'basis_heart_rate', 'basis_air_temperature']
            elif dataset == 'arises':
                attri_list = [
            'meal', 'bolus', 'basal', 'correction_bolus',
            'finger_stick', 'EDA', 'SCL', 'SCR', 'HR', 'TEMP', 'ACC', 'RMSSD', 'SDNN',
            'medianNNI', 'CVNNI', 'CVSD', 'pNNX', 'meanHR', 'minHR', 'maxHR', 'VLF',
            'LF', 'HF', 'LHR', ]
            elif dataset == 'shanghai_t1dm' or dataset == 'shanghai_t2dm':
                attri_list = ['finger_stick', 'blood_ketone', 'meal','insulin_dose_sc', 'hypo_agents', 'insulin_bolus', 'basal','insulin_iv']

            run_exp(dataset, version, method, seed, pred_window, n_prev, seq_len, attri_list, interval)

linear_regression shanghai_t1dm 1
rmse, 22.57032, 23.53105
mape, 11.46699, 6.37660
mae, 15.81135, 13.32423
grmse, 26.33403, 26.77370
time_lag, -2.50000, 5.59017
[99.67122712009218, 10.902743674924825, 13.594591838157324, 24.10189044549981, 18.296835804025054, 15.975279307423557, 13.219081865478062, 12.320617168870843, 19.31509451119933, 11.102636254481023, 15.012143623638758, 17.331676012671224]
linear_regression shanghai_t1dm 10
rmse, 22.57032, 23.53105
mape, 11.46699, 6.37660
mae, 15.81135, 13.32423
grmse, 26.33403, 26.77370
time_lag, -2.50000, 5.59017
[99.67122712009218, 10.902743674924825, 13.594591838157324, 24.10189044549981, 18.296835804025054, 15.975279307423557, 13.219081865478062, 12.320617168870843, 19.31509451119933, 11.102636254481023, 15.012143623638758, 17.331676012671224]
linear_regression shanghai_t1dm 20
rmse, 22.57032, 23.53105
mape, 11.46699, 6.37660
mae, 15.81135, 13.32423
grmse, 26.33403, 26.77370
time_lag, -2.50000, 5.59017
[99.67122712009218, 10.902743674924825,

<Figure size 2000x1500 with 0 Axes>

<Figure size 2000x1500 with 0 Axes>

<Figure size 2000x1500 with 0 Axes>

<Figure size 2000x1500 with 0 Axes>

...

<Figure size 2000x1500 with 0 Axes>

<Figure size 2000x1500 with 0 Axes>

<Figure size 2000x1500 with 0 Axes>

<Figure size 2000x1500 with 0 Axes>

<Figure size 2000x1500 with 0 Axes>